In [ ]:
!pip install openai -q
!pip install azure-storage-blob
!pip install langchain
!pip install tiktoken
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import openai
from langchain.document_loaders import AzureBlobStorageContainerLoader
import numpy as np
import os

In [ ]:
openai.api_key = ''
deployment_name = 'text-davinci-003'

In [ ]:
print('Sending a test completion job')
start_phrase = 'Write a tagline for an ice cream shop. '
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)

Sending a test completion job
Write a tagline for an ice cream shop. The best ice cream in town!


In [ ]:
loaders = AzureBlobStorageContainerLoader(conn_str="DefaultEndpointsProtocol=https;AccountName=openaiblob1;EndpointSuffix=core.windows.net",container="manyonepagefilesonsports-ishita")
documents = loaders.load()
print(f'You have {len(documents)} documents in your data')
text_to_summarize2 = []
for doc in documents:
  text_to_summarize2.append(doc.page_content)

print(text_to_summarize2[0])

In [ ]:
file_names = [os.path.basename(doc.metadata['source']) for doc in documents]
print(file_names)

['cricket summary.docx', 'football summary.docx', 'hockey summary.docx', 'tennis summary.docx']


In [ ]:
import math

words = text_to_summarize2[0].split(" ")
print(len(words))
split_len = len(words)//2500 + 1
chunks = np.array_split(words, split_len)
print(len(chunks), len(chunks[0]))

1186
1 1186


In [ ]:
documents, text_to_summarize2

([Document(page_content='Benefits For Out-Of-Network Or Non-Contracted Providers \n\nHOW PROVIDERS AFFECT YOUR COSTS: \n\nBenefits For Out-Of-Network Or Non-Contracted Providers\n\nOne of the important considerations when choosing a health insurance plan is understanding the differences between in-network and out-of-network providers. When you receive health care from an in-network provider, your insurance plan will generally cover a greater portion of your expenses. However, when you receive health care from an out-of-network provider, you may be required to pay a greater portion of the costs. Under Northwind Standard, you may receive care from out-of-network providers, but you will be responsible for a greater portion of the costs. To make sure you are getting the best value for your health care expenses, it is important to understand the differences between in-network and out-of-network providers and how they affect your costs. In-Network: In-network providers are those who have con

In [ ]:
count=0
def recursive_summarize(text_to_summarize, target_word_count=200):
    words = text_to_summarize.split(" ")
    split_len = len(words)//2500 + 1
    chunks = np.array_split(words, split_len)
    #chunks = np.array_split(words, 1)
    summary_responses = []
    full_summary = ""

    for chunk in chunks:
        sentences = ' '.join(list(chunk))
        prompt = f"{sentences}\n\ntl;dr:"
        response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.3,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=1
        )

        response_text = response["choices"][0]["text"]
        summary_responses.append(response_text+'\n\n')

    full_summary = "".join(summary_responses)
    word_count = len(full_summary.split())
    print(word_count)
    if word_count > target_word_count:
    #     print(++count)
        return recursive_summarize(full_summary, target_word_count)

    return full_summary

#Summary of each doc
no_of_docs = len(text_to_summarize2)
final_summary = []
for i in range(no_of_docs):
  initial_text_to_summarize = text_to_summarize2[i]
  final_summary.append(recursive_summarize(initial_text_to_summarize, target_word_count=50))

# Example usage
# initial_text_to_summarize = text_to_summarize2[0]
# final_summary = recursive_summarize(initial_text_to_summarize, target_word_count=50)


84
64
63
45
67
54
43


KeyboardInterrupt: ignored

In [ ]:
final_summary #Summary of each doc

['\n\nIn-network providers are generally less expensive than out-of-network providers, and Northwind Standard offers coverage for a variety of in-network providers. To get the best value for your health care services, it’s important to understand how providers can affect your costs and to research providers in advance.\n\n',
 '\n\nThe Northwind Standard insurance plan covers the cost of allergy testing and treatment, including skin testing, blood testing, immunotherapy, and prescription and non-prescription medications. Allergy testing and treatment are not covered for cosmetic purposes.\n\n',
 ' You pay 20% of the cost for in-network services, with some exceptions. For out-of-network services, you pay 40%.\n\n',
 '\n\nWhen choosing a provider, be sure to confirm that they are in-network with your health insurance plan. This will help you avoid balance billing and unexpected costs. You can also ask your provider about their policy on balance billing and discounts for services.\n\n',
 "

In [ ]:
def recursive_summarize(text_to_summarize, target_word_count=500):
    words = text_to_summarize.split(" ")
    split_len = len(words)//2500 + 1
    chunks = np.array_split(words, split_len)
    #chunks = np.array_split(words, 1)
    summary_responses = []
    full_summary = ""

    for chunk in chunks:
        sentences = ' '.join(list(chunk))
        prompt = f"Summarize and compare information from the following chunks preserving the important subheadings: {sentences}\n\n"
        response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.3,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=1
        )

        response_text = response["choices"][0]["text"]
        summary_responses.append(response_text+'\n\n')

    full_summary = "".join(summary_responses)
    word_count = len(full_summary.split())
    print(word_count)
    if word_count > target_word_count:
    #     print(++count)
        return recursive_summarize(full_summary, target_word_count)

    return full_summary


In [ ]:
initial_text_to_summarize = text_to_summarize2
final_summary = recursive_summarize(initial_text_to_summarize, target_word_count=500)
print(final_summary)